In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
import commons as cmns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from IPython.display import Audio
sound_file = 'beep.wav'

In [2]:
data=pd.read_csv('BTC_Data_736_features_raw.csv',sep=',')

In [3]:
data.interpolate(axis=0,inplace=True)

In [4]:
interval1 = (data['Date'] >= '2013/04/01') & (data['Date'] <= '2016/04/01')
interval2=(data['Date'] >= '2013/04/01') & (data['Date'] <= '2017/04/01')
interval3=(data['Date'] >= '2013/04/01') & (data['Date'] <= '2019/12/31')

In [5]:
one=data['priceUSD'].shift(-1,fill_value=1)
seven=data['priceUSD'].shift(-7,fill_value=1)
fourteen=data['priceUSD'].shift(-14,fill_value=1)
thirty=data['priceUSD'].shift(-30,fill_value=1)
sixty=data['priceUSD'].shift(-60,fill_value=1)
ninety=data['priceUSD'].shift(-90,fill_value=1)

In [6]:
df=data.loc[interval1]

In [7]:
#df['priceUSD']=one.loc[interval3]

In [8]:
df.shape

(1097, 737)

In [8]:
X_raw=df.iloc[:,2:17]

In [9]:
y=df.iloc[:,1:2]
y=np.ravel(y)

In [10]:
features_list=[]
technical_indicators=['sma','ema','wma','trx','mom','std','var','rsi','roc']
periods=['3','7','14','30','90']
for i in technical_indicators:
    for j in periods:
        string=str(j)+str(i)
        X=df.filter(like=string,axis=1)
        X=SimpleImputer(missing_values=0,strategy='most_frequent').fit_transform(X)
        X=pd.DataFrame(X)
        X.columns=df.filter(like=string,axis=1).columns
        rf1=RandomForestRegressor(random_state=7,n_jobs=-1)
        rfecv=RFECV(rf1,step=0.9,min_features_to_select=1,verbose=1,cv=5,scoring='neg_mean_absolute_error', n_jobs=-1)
        rfecv.fit(X,y)
        if rfecv.n_features_>1:
            rf1.fit(X,y)
            maximp=rf1.feature_importances_.max()
            for x in range(len(rf1.feature_importances_)):
                if maximp==rf1.feature_importances_[x]:
                    new_features = X.columns[x]
        else:
            mask = rfecv.get_support()
            new_features = X.columns[mask]
        features_list.append(str(new_features))
        print(string+': '+new_features)
Audio(sound_file, autoplay=True)

Fitting estimator with 16 features.
Fitting estimator with 2 features.
Index(['3sma: price3smaUSD'], dtype='object')
Fitting estimator with 16 features.
Fitting estimator with 2 features.
Index(['7sma: price7smaUSD'], dtype='object')
Fitting estimator with 16 features.
Fitting estimator with 2 features.
Index(['14sma: price14smaUSD'], dtype='object')
30sma: transactionvalue30smaUSD
90sma: mediantransactionvalue90smaUSD
Fitting estimator with 16 features.
Fitting estimator with 2 features.
Index(['3ema: price3emaUSD'], dtype='object')
Fitting estimator with 16 features.
Fitting estimator with 2 features.
Index(['7ema: price7emaUSD'], dtype='object')
Fitting estimator with 16 features.
Fitting estimator with 2 features.
Index(['14ema: price14emaUSD'], dtype='object')
Fitting estimator with 16 features.
30ema: transactionvalue30emaUSD
90ema: transactionvalue90emaUSD
Fitting estimator with 16 features.
Fitting estimator with 2 features.
Index(['3wma: price3wmaUSD'], dtype='object')
Fitting

In [11]:
l1=[]
for j in range(len(features_list)):
    result1 = re.search("'(.*)'],", features_list[j])
    if result1!=None:
        l1.append(result1.group(1))
for i in range(len(features_list)):
    result2 = re.search('.*',features_list[i])
    if len(result2.group(0))<33:
        l1.append(result2.group(0))

In [12]:
l1.sort()

In [13]:
l1

['difficulty14mom',
 'difficulty14roc',
 'difficulty14rsi',
 'difficulty14trx',
 'difficulty30mom',
 'difficulty30roc',
 'difficulty30rsi',
 'difficulty3trx',
 'difficulty7roc',
 'difficulty7trx',
 'difficulty90roc',
 'difficulty90rsi',
 'fee_to_reward30trxUSD',
 'fee_to_reward90trxUSD',
 'hashrate3std',
 'hashrate3var',
 'hashrate7mom',
 'mediantransactionvalue90smaUSD',
 'mining_profitability3mom',
 'mining_profitability7rsi',
 'mining_profitability7std',
 'mining_profitability7var',
 'price14emaUSD',
 'price14smaUSD',
 'price14wmaUSD',
 'price30wmaUSD',
 'price3emaUSD',
 'price3rsiUSD',
 'price3smaUSD',
 'price3wmaUSD',
 'price7emaUSD',
 'price7smaUSD',
 'price7wmaUSD',
 'price90momUSD',
 'sentinusd30stdUSD',
 'sentinusd30varUSD',
 'sentinusd90stdUSD',
 'sentinusd90varUSD',
 'transactionfees3rocUSD',
 'transactionvalue14stdUSD',
 'transactionvalue14varUSD',
 'transactionvalue30emaUSD',
 'transactionvalue30smaUSD',
 'transactionvalue90emaUSD',
 'transactionvalue90wmaUSD']

In [14]:
df_reduced=df[l1]

In [15]:
df_reduced.head()

difficulty14mom  difficulty14roc  difficulty14rsi  difficulty14trx  \
989        1848179.0           38.125           99.825            1.930   
990        1848179.0           38.125           99.825            1.948   
991        1848179.0           38.125           99.825            1.943   
992        1848179.0           38.125           99.825            1.917   
993        2157411.0           44.504           99.867            1.888   

     difficulty30mom  difficulty30roc  difficulty30rsi  difficulty3trx  \
989        2327950.0           53.297           97.889           0.845   
990        2327950.0           53.297           97.889           0.520   
991        2327950.0           53.297           97.889           0.314   
992        2327950.0           53.297           97.889           0.187   
993        2637182.0           60.377           98.186           0.689   

     difficulty7roc  difficulty7trx  ...  sentinusd30varUSD  \
989           0.000           2.593  ...       2.698640e+15   
990           0.000           2.333  ...       3.541060e+15   
991           0.000           2.064  ...       4.321521e+15   
992           0.000           1.801  ...       5.132381e+15   
993           4.618           1.634  ...       5.734210e+15   

     sentinusd90stdUSD  sentinusd90varUSD  transactionfees3rocUSD  \
989           88825639       1.972499e+15                  -8.219   
990          100243842       2.512207e+15                  10.080   
991          110663079       3.061579e+15                  42.924   
992          119905933       3.594358e+15                  81.728   
993          126900119       4.025910e+15                  15.109   

     transactionvalue14stdUSD  transactionvalue14varUSD  \
989                    1325.0                  438600.0   
990                    1673.0                  699432.0   
991                    1939.0                  940396.0   
992                    2083.0                 1085025.0   
993                    2108.0                 1110469.0   

     transactionvalue30emaUSD  transactionvalue30smaUSD  \
989                    2019.0                    1883.0   
990                    2172.0                    2012.0   
991                    2321.0                    2144.0   
992                    2446.0                    2244.0   
993                    2568.0                    2341.0   

     transactionvalue90emaUSD  transactionvalue90wmaUSD  
989                    1335.0                    1380.0  
990                    1403.0                    1454.0  
991                    1470.0                    1528.0  
992                    1531.0                    1597.0  
993                    1593.0                    1667.0  

[5 rows x 45 columns]

In [16]:
X=cmns.drop_high_vif(df_reduced,thresh=5)

Remaining variables:
Index(['difficulty14mom', 'difficulty14roc', 'difficulty14rsi',
       'difficulty14trx', 'difficulty30mom', 'difficulty30roc',
       'difficulty30rsi', 'difficulty3trx', 'difficulty7roc', 'difficulty7trx',
       'difficulty90roc', 'difficulty90rsi', 'fee_to_reward30trxUSD',
       'fee_to_reward90trxUSD', 'hashrate3std', 'hashrate3var', 'hashrate7mom',
       'mediantransactionvalue90smaUSD', 'mining_profitability3mom',
       'mining_profitability7rsi', 'mining_profitability7std',
       'mining_profitability7var', 'price14emaUSD', 'price14smaUSD',
       'price14wmaUSD', 'price30wmaUSD', 'price3emaUSD', 'price3rsiUSD',
       'price3smaUSD', 'price3wmaUSD', 'price7emaUSD', 'price7smaUSD',
       'price7wmaUSD', 'price90momUSD', 'sentinusd30stdUSD',
       'sentinusd30varUSD', 'sentinusd90stdUSD', 'sentinusd90varUSD',
       'transactionfees3rocUSD', 'transactionvalue14stdUSD',
       'transactionvalue14varUSD', 'transactionvalue30emaUSD',
       'transactionva

In [17]:
X.shape

(1097, 45)

In [18]:
cmns.vif_values(X)

/home/mudassir/btcpaper/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


difficulty14mom                   1.535715
difficulty14roc                   0.560667
difficulty14rsi                   0.039893
difficulty14trx                   0.430588
difficulty30mom                   1.389658
difficulty30roc                   0.516858
difficulty30rsi                   0.021075
difficulty3trx                    0.672958
difficulty7roc                    0.675116
difficulty7trx                    0.473492
difficulty90roc                   0.604475
difficulty90rsi                   0.010271
fee_to_reward30trxUSD             1.065212
fee_to_reward90trxUSD             1.063052
hashrate3std                      3.031699
hashrate3var                      4.458547
hashrate7mom                      1.027581
mediantransactionvalue90smaUSD    0.216332
mining_profitability3mom          0.998009
mining_profitability7rsi          0.042581
mining_profitability7std          0.928430
mining_profitability7var          0.985263
price14emaUSD                     0.262024
price14smaU

In [19]:
Raw=cmns.drop_high_vif(X_raw,thresh=10)

dropping 'hashrate' at index: 4
dropping 'sentbyaddress' at index: 2
dropping 'top100cap' at index: 11
dropping 'transactions' at index: 0
dropping 'size' at index: 0
dropping 'transactionvalueUSD' at index: 6
dropping 'activeaddresses' at index: 7
dropping 'median_transaction_feeUSD' at index: 4
Remaining variables:
Index(['difficulty', 'mining_profitability', 'sentinusdUSD',
       'transactionfeesUSD', 'confirmationtime', 'mediantransactionvalueUSD',
       'fee_to_rewardUSD'],
      dtype='object')


In [20]:
#cmns.vif_values(X_raw)
cmns.vif_values(Raw)

difficulty                   4.399881
mining_profitability         1.876837
sentinusdUSD                 1.638621
transactionfeesUSD           9.419428
confirmationtime             5.818466
mediantransactionvalueUSD    6.841771
fee_to_rewardUSD             7.305746
dtype: float64

In [21]:
Raw['Date']=df['Date']

/home/mudassir/btcpaper/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [22]:
X['Date']=df['Date']

In [23]:
df_merged=Raw.merge(X.drop_duplicates(subset='Date'),how='left')

In [24]:
df_merged.shape

(1097, 53)

In [25]:
df_merged.drop(columns='Date',inplace=True)

In [26]:
#df_merged['priceUSD']=np.ravel(y)

In [27]:
X=cmns.drop_high_vif(df_merged,thresh=10)

dropping 'sentinusd30stdUSD' at index: 41
dropping 'sentinusd90stdUSD' at index: 42
Remaining variables:
Index(['difficulty', 'mining_profitability', 'sentinusdUSD',
       'transactionfeesUSD', 'confirmationtime', 'mediantransactionvalueUSD',
       'fee_to_rewardUSD', 'difficulty14mom', 'difficulty14roc',
       'difficulty14rsi', 'difficulty14trx', 'difficulty30mom',
       'difficulty30roc', 'difficulty30rsi', 'difficulty3trx',
       'difficulty7roc', 'difficulty7trx', 'difficulty90roc',
       'difficulty90rsi', 'fee_to_reward30trxUSD', 'fee_to_reward90trxUSD',
       'hashrate3std', 'hashrate3var', 'hashrate7mom',
       'mediantransactionvalue90smaUSD', 'mining_profitability3mom',
       'mining_profitability7rsi', 'mining_profitability7std',
       'mining_profitability7var', 'price14emaUSD', 'price14smaUSD',
       'price14wmaUSD', 'price30wmaUSD', 'price3emaUSD', 'price3rsiUSD',
       'price3smaUSD', 'price3wmaUSD', 'price7emaUSD', 'price7smaUSD',
       'price7wmaUSD', 'pr

In [28]:
X.shape

(1097, 50)

In [29]:
cmns.vif_values(X)

difficulty                        1.383727
mining_profitability              0.880295
sentinusdUSD                      1.100001
transactionfeesUSD                0.341585
confirmationtime                  0.017395
mediantransactionvalueUSD         0.357440
fee_to_rewardUSD                  0.418142
difficulty14mom                   1.535819
difficulty14roc                   0.563529
difficulty14rsi                   0.040250
difficulty14trx                   0.432463
difficulty30mom                   1.683284
difficulty30roc                   0.518539
difficulty30rsi                   0.021267
difficulty3trx                    0.676725
difficulty7roc                    0.678463
difficulty7trx                    0.476047
difficulty90roc                   0.606164
difficulty90rsi                   0.010358
fee_to_reward30trxUSD             1.076395
fee_to_reward90trxUSD             1.065742
hashrate3std                      3.031728
hashrate3var                      8.460534
hashrate7mo

In [30]:
if 'priceUSD' in X.columns:
    X.drop(columns='priceUSD',inplace=True)

In [31]:
rf_final=RandomForestRegressor(random_state=7,n_jobs=-1)

In [32]:
rf_final.fit(X,y)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=7, verbose=0, warm_start=False)

In [33]:
imp_feat=cmns.feature_importance(X.columns,rf_final.feature_importances_)

In [34]:
cmns.plot_feature_importance(imp_feat,'feature importance')

In [35]:
imp_feat[imp_feat['feature_importance']>0]


feature  feature_importance
0                     price3wmaUSD        6.802920e-01
1                     price3emaUSD        2.398325e-01
2                     price3smaUSD        1.981879e-02
3                    price90momUSD        1.418388e-02
4                    price14emaUSD        9.828864e-03
5                     price7wmaUSD        8.310717e-03
6                     price7smaUSD        7.243063e-03
7                    price30wmaUSD        7.107601e-03
8                    price14wmaUSD        4.170122e-03
9                    price14smaUSD        3.505285e-03
10                    price7emaUSD        2.442800e-03
11                      difficulty        1.154498e-03
12                    price3rsiUSD        4.644588e-04
13                    hashrate3std        1.312194e-04
14              transactionfeesUSD        1.186910e-04
15                    hashrate3var        9.591073e-05
16       mediantransactionvalueUSD        8.886320e-05
17          transactionfees3rocUSD        7.842732e-05
18               sentinusd30varUSD        7.473594e-05
19  mediantransactionvalue90smaUSD        7.390197e-05
20        mining_profitability7rsi        6.847390e-05
21                fee_to_rewardUSD        6.414045e-05
22           fee_to_reward30trxUSD        6.346667e-05
23        mining_profitability3mom        6.283701e-05
24                    sentinusdUSD        5.309390e-05
25                 difficulty90roc        4.947140e-05
26                 difficulty14mom        4.279740e-05
27                 difficulty14trx        4.276777e-05
28               sentinusd90varUSD        4.039890e-05
29                  difficulty7trx        3.969822e-05
30           fee_to_reward90trxUSD        3.445238e-05
31            mining_profitability        3.437650e-05
32                confirmationtime        3.372067e-05
33                    hashrate7mom        3.286098e-05
34                 difficulty30roc        3.094108e-05
35                  difficulty3trx        3.071261e-05
36        mining_profitability7var        2.893369e-05
37        transactionvalue30emaUSD        2.776580e-05
38        transactionvalue90wmaUSD        2.609145e-05
39                 difficulty14roc        2.486188e-05
40                  difficulty7roc        2.442656e-05
41        transactionvalue30smaUSD        2.418162e-05
42        mining_profitability7std        2.074669e-05
43        transactionvalue14varUSD        2.050543e-05
44        transactionvalue14stdUSD        1.902493e-05
45        transactionvalue90emaUSD        1.699109e-05
46                 difficulty30mom        1.223999e-05
47                 difficulty90rsi        1.157477e-05
48                 difficulty30rsi        1.377589e-06
49                 difficulty14rsi        7.049372e-07

In [71]:
cols=[0,3,5,7,11,13,14,16,18,19,20,21]

In [72]:
#selected=imp_feat[imp_feat['feature_importance']>0.001]
selected=imp_feat.filter(cols,axis=0)

In [73]:
selected

feature  feature_importance
0                     price3wmaUSD            0.680292
3                    price90momUSD            0.014184
5                     price7wmaUSD            0.008311
7                    price30wmaUSD            0.007108
11                      difficulty            0.001154
13                    hashrate3std            0.000131
14              transactionfeesUSD            0.000119
16       mediantransactionvalueUSD            0.000089
18               sentinusd30varUSD            0.000075
19  mediantransactionvalue90smaUSD            0.000074
20        mining_profitability7rsi            0.000068
21                fee_to_rewardUSD            0.000064

In [74]:
selected.feature_importance.sum()

0.7116687102566427

In [75]:
X_high=X[selected.feature]

In [76]:
X_high.shape

(1097, 12)

In [77]:
X_high.sort_index(axis=1,inplace=True)

/home/mudassir/btcpaper/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [78]:
X_high['priceUSD']=np.ravel(y)

/home/mudassir/btcpaper/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [79]:
corr = X_high.corr()
# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=np.bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(15, 8))

sns.heatmap(corr,mask=mask, cmap='coolwarm',annot=True, fmt='.2f')

In [80]:
X_high=X_high[X_high['priceUSD']!=1]

In [85]:
from sklearn.ensemble import IsolationForest
outlier=IsolationForest(contamination=0.9)
price=X_high['priceUSD']
price=price.values
price=np.reshape(price,(-1,1))
outlier.fit(price)
outs=outlier.predict(price)
X_high['outlier']=pd.DataFrame(zip(outs))
X_high=X_high.query('outlier != -1')
X_high.reset_index(drop=True,inplace=True)
X_high.drop(columns=['outlier'],inplace=True)

In [86]:
X_high.to_csv('reg_interval1.csv',sep=',',index=False)

In [87]:
X_high.columns

Index(['difficulty', 'fee_to_rewardUSD', 'hashrate3std',
       'mediantransactionvalue90smaUSD', 'mediantransactionvalueUSD',
       'mining_profitability7rsi', 'price30wmaUSD', 'price3wmaUSD',
       'price7wmaUSD', 'price90momUSD', 'sentinusd30varUSD',
       'transactionfeesUSD', 'priceUSD'],
      dtype='object')